In [ ]:
import numpy as np
import random
import tensorflow as tf
import sklearn.datasets
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
DIM = 512
CRITIC_ITERS = 1
BATCH_SIZE = 256
ITERS = 100000
DATASET = '8gaussians'

In [ ]:
def spectral_norm(w, iteration=1):
    w_shape = w.shape.as_list()
    w = tf.reshape(w, [-1, w_shape[-1]])

    u = tf.get_variable('u', [1, w_shape[-1]], initializer=tf.random_normal_initializer(), trainable=False)

    u_hat = u
    v_hat = None
    for i in range(iteration):
        v_ = tf.matmul(u_hat, tf.transpose(w))
        v_hat = tf.nn.l2_normalize(v_)

        u_ = tf.matmul(v_hat, w)
        u_hat = tf.nn.l2_normalize(u_)

    u_hat = tf.stop_gradient(u_hat)
    v_hat = tf.stop_gradient(v_hat)

    sigma = tf.matmul(tf.matmul(v_hat, w), tf.transpose(u_hat))

    with tf.control_dependencies([u.assign(u_hat)]):
        w_norm = w / sigma
        w_norm = tf.reshape(w_norm, w_shape)

    return w_norm

def snlinear(inputs, input_dim, output_dim, activation, name):
    with tf.variable_scope(name):
        weights = tf.get_variable(
            'kernel',
            [input_dim, output_dim],
            initializer=tf.random_normal_initializer(0., output_dim ** -0.5)
        )
        biases = tf.get_variable('bias', [output_dim])
        outputs = tf.matmul(inputs, spectral_norm(weights)) + biases
        if activation:
            outputs = activation(outputs)

        return outputs

def Generator(n_samples):
    with tf.variable_scope('Generator', reuse=tf.AUTO_REUSE):
        noise = tf.random.normal([n_samples, 2])
        outputs = tf.layers.dense(noise, DIM, activation=tf.nn.relu, name='Generator.1')
        outputs = tf.layers.dense(outputs, DIM, activation=tf.nn.relu, name='Generator.2')
        outputs = tf.layers.dense(outputs, DIM, activation=tf.nn.relu, name='Generator.3')
        outputs = tf.layers.dense(outputs, 2, activation=None, name='Generator.4')
        return outputs

def Discriminator(inputs):
    with tf.variable_scope('Discriminator', reuse=tf.AUTO_REUSE):
        outputs = snlinear(inputs, 2, DIM, activation=tf.nn.relu, name='Discriminator.1')
        outputs = snlinear(outputs, DIM, DIM, activation=tf.nn.relu, name='Discriminator.2')
        outputs = snlinear(outputs, DIM, DIM, activation=tf.nn.relu, name='Discriminator.3')
        outputs = snlinear(outputs, DIM, 1, activation=None, name='Discriminator.4')
        return outputs

In [ ]:
real_data = tf.placeholder(tf.float32, shape=[None, 2])

fake_data = Generator(BATCH_SIZE)
disc_real = Discriminator(real_data)
disc_fake = Discriminator(fake_data)

disc_loss = tf.reduce_mean(disc_fake) - tf.reduce_mean(disc_real)
gen_loss = -tf.reduce_mean(disc_fake)

In [ ]:
disc_params = tf.trainable_variables('Discriminator')
gen_params = tf.trainable_variables('Generator')

disc_train_op = tf.train.AdamOptimizer(
    learning_rate=3e-4,
    beta1=0.0,
    beta2=0.9
).minimize(disc_loss, var_list=disc_params)

gen_train_op = tf.train.AdamOptimizer(
    learning_rate=1e-4,
    beta1=0.0,
    beta2=0.9
).minimize(gen_loss, var_list=gen_params)

In [ ]:
def create_real_data():
    if DATASET == '25gaussians':
        positions = 2 * np.random.choice(range(-2, 3), size=BATCH_SIZE*2)
        noise = 0.05 * np.random.randn(BATCH_SIZE*2)
        yield np.reshape(positions + noise, [BATCH_SIZE, 2]) / 2.828
    elif DATASET == 'swissroll':
        data = sklearn.datasets.make_swiss_roll(
            n_samples=BATCH_SIZE, 
            noise=0.25
        )[0]
        data = data.astype('float32')[:, [0, 2]]
        data /= 7.5
        yield data
    elif DATASET == '8gaussians':
        scale = 2.
        centers = [
            (1,0),
            (-1,0),
            (0,1),
            (0,-1),
            (1./np.sqrt(2), 1./np.sqrt(2)),
            (1./np.sqrt(2), -1./np.sqrt(2)),
            (-1./np.sqrt(2), 1./np.sqrt(2)),
            (-1./np.sqrt(2), -1./np.sqrt(2))
        ]
        centers = [(scale*x,scale*y) for x,y in centers]
        dataset = []
        for i in range(BATCH_SIZE):
            point = np.random.randn(2)*.02
            center = random.choice(centers)
            point[0] += center[0]
            point[1] += center[1]
            dataset.append(point)
        dataset = np.array(dataset, dtype='float32')
        dataset /= 1.414
        yield dataset

def generate_image():
    N_POINTS = 128
    RANGE = 3
    points = np.zeros((N_POINTS, N_POINTS, 2), dtype='float32')
    points[:,:,0] = np.linspace(-RANGE, RANGE, N_POINTS)[:,None]
    points[:,:,1] = np.linspace(-RANGE, RANGE, N_POINTS)[None,:]
    points = points.reshape((-1,2))
    disc_map = sess.run(disc_real, feed_dict={real_data:points})
    x = y = np.linspace(-RANGE, RANGE, N_POINTS)
    plt.contour(x, y, disc_map.reshape((len(x), len(y))).transpose())

    fake_data_samples = sess.run(fake_data)
    real_data_samples = next(create_real_data())

    plt.scatter(real_data_samples[:, 0], real_data_samples[:, 1], c='orange', marker='+')
    plt.scatter(fake_data_samples[:, 0], fake_data_samples[:, 1], c='green', marker='+')
    plt.show()

In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [ ]:
%%time

for i in range(ITERS+1):
    for _ in range(CRITIC_ITERS):
        fd = {real_data: next(create_real_data())}
        _disc_loss, _ = sess.run([disc_loss, disc_train_op], fd)

    sess.run(gen_train_op)

    if i % 10000 == 0:
        print(i, _disc_loss)
        generate_image()
